In [1]:
import os
import json
import dateparser
import datetime
from ladder_page import LadderPage
from history_page import HistoryPage
from retry_decorator import retry

In [2]:
@retry(Exception, tries=10, timeout_secs=30)
def get_new_games(player, server, last_bq_date=None):
    start_date_for_db = datetime.datetime(2018, 1, 1)
    def no_more_dates():
        if last_bq_date is None:
            return False
        else:
            return current_date <= last_bq_date
        
    def no_more_pages():
        return page >= next_page
    
    new_games = []
    page = 1
    
    while True:
        history_page = HistoryPage(player, server, page)
        games = history_page.games
        next_page = history_page.next_page

        for d in games:
            current_date = dateparser.parse(d.get('date'))
            if no_more_dates() or (current_date < start_date_for_db):
                return new_games
            else:
                new_games.append(d)

        if no_more_pages():
            return new_games
        else:
            page = next_page

In [4]:
server= "azeroth"

In [5]:
ladder_page = LadderPage(server, page=page)

NameError: name 'page' is not defined

In [6]:
servers = ['azeroth', 'northrend', 'lorderon']
for server in servers:
    dir_ = './data_backfill/partial/{}'.format(server)
    if not os.path.exists(dir_):
        os.makedirs(dir_)
    
    page = 1
    # Loop over the first 5 ladder pages.
    for page in range(1, 10):
        ladder_page = LadderPage(server, page=page)
        print('\n{}'.format(page))
        
        # On each page select a player and get their games.
        for row in ladder_page.rows:
            games = []
            player = row.get('player')
            
            file_path = '{}/{}.json'.format(dir_, player)
            if os.path.exists(file_path):
#                 print('{} - Already exists'.format(player))
                continue

            games = get_new_games(player, server)
            print('\t{} - {}'.format(player, len(games)))
            with open(file_path, 'w') as f:
                json.dump(games, f)


1


KeyboardInterrupt: 

In [ ]:
def wash_player_name(player):
    return player.replace('|c', '%7')

In [7]:
import requests
from bs4 import BeautifulSoup
from helpers import wash_player_name
from config import data_positions_history
from urllib.parse import parse_qs
import pandas as pd
import json


class HistoryPage:
    def __init__(self, player, server, page=1):
        self.player = player
        self.server = server.title()
        self.page = page
        self.params = {'PlayerName': self.player, 'Gateway': self.server, 'PageNo': page}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-logged-games.aspx'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def __repr__(self):
        return '{}@{}'.format(self.player, self.server)

    def _validate(self):
        self.validate_server()
        self.validate_player()

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params)
        except requests.exceptions.RequestException as e:
            print(e)

        return BeautifulSoup(r.content, 'lxml')

    @property
    def game_containers(self):
        return self.soup.find('table', id='tblGames').find_all('tr', class_='rankingRow')[1:]

    def games(self):
        for game in self.game_containers:
            yield Game(self.player, game).parse()

    @property
    def next_page(self):
        url = self.soup.find(text='Next\xa0Page').parent.get('href')
        if url is not None:
            next_page = parse_qs(url).get('PageNo')[0]
            return int(next_page)


class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup

    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        game_id = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(game_id)
        data['team_one'].append(self.player)

        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]

        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']

        return data

    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

if __name__ == '__main__':
    players = [
        {
            'player': 'Rellik',
            'server': 'northrend'
        }
    ]

    print('-- Testing --')
    for player in players:
        page = 1
        data_all = []

        while True:
                history_page = HistoryPage(player.get('player'), player.get('server'), page)
                data = list(history_page.games())
                data_all.extend(data)
                next_page = history_page.next_page
                if page >= next_page:
                        break
                page = next_page

        df = pd.DataFrame(data_all)
        print(df.shape)
        print(df.head(3))

    data = df.to_dict(orient='records')
    print(data)
    with open('./data_backfill/{}.json'.format(player.get('player')), 'w') as f:
        json.dump(data, f)


-- Testing --
(53, 8)
                  date   game_id  game_length game_type                map  \
0  2019-02-04 18:50:00  79855255            7      Solo     Northern Isles   
1  2019-02-04 18:39:00  79855180            7      Solo           Amazonia   
2  2019-02-04 16:17:00  79854382           20      Solo  Plunder Isle_ L V   

   team_one         team_two    winner  
0  [Rellik]     [MegamanEXE]  [Rellik]  
1  [Rellik]  [TimeIsDancing]  [Rellik]  
2  [Rellik]     [Malicious-]  [Rellik]  
[{'date': '2019-02-04 18:50:00', 'game_id': 79855255, 'game_length': 7, 'game_type': 'Solo', 'map': 'Northern Isles', 'team_one': ['Rellik'], 'team_two': ['MegamanEXE'], 'winner': ['Rellik']}, {'date': '2019-02-04 18:39:00', 'game_id': 79855180, 'game_length': 7, 'game_type': 'Solo', 'map': 'Amazonia', 'team_one': ['Rellik'], 'team_two': ['TimeIsDancing'], 'winner': ['Rellik']}, {'date': '2019-02-04 16:17:00', 'game_id': 79854382, 'game_length': 20, 'game_type': 'Solo', 'map': 'Plunder Isle_ L V'

In [8]:
class Game:
    def __init__(self, player, soup):
        self.player = player
        self.soup = soup
        
    
    def parse(self):
        values = self.soup.find_all(['td'])
        values = [x.get_text().strip() for x in values]
        data = self.parse_values(values)
        id_ = self.soup.find_all(['td'])[0].a.get('href').split('&GameID=')[1]
        data['game_id'] = int(id_)
        data['team_one'].append(self.player)
        
        for players in ['team_one', 'team_two']:
            data[players] = [wash_player_name(player) for player in data[players]]
            
        if data['winner'] == 'Win':
            data['winner'] = data['team_one']
        else:
            data['winner'] = data['team_two']
            
        
        return data
     
    
    @staticmethod
    def parse_values(values):
        data = {}
        for field, meta_data in data_positions_history.items():   
            i = meta_data['position']
            v = values[i]
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            data[field] = value

        return data

In [9]:
history_page = HistoryPage('followgrubby', 'northrend')
print(history_page)

followgrubby@Northrend


In [10]:
data = list(history_page.games())
df = pd.DataFrame(data)
df

,date,game_id,game_length,game_type,map,team_one,team_two,winner
0,2019-02-03 01:22:00,79845695,11,Solo,Twisted Meadows,[followgrubby],[pwarrior],[pwarrior]
1,2019-02-03 01:08:00,79845644,12,Solo,Concealed Hill,[followgrubby],[Uberelf],[followgrubby]
2,2019-02-03 00:52:00,79845577,12,Solo,Last Refuge,[followgrubby],[atm],[followgrubby]
3,2019-02-03 00:37:00,79845522,9,Solo,Northern Isles,[followgrubby],[Critical],[followgrubby]
4,2019-02-02 22:57:00,79844958,10,Solo,Twisted Meadows,[followgrubby],[Alchh],[followgrubby]
5,2019-02-02 22:40:00,79844851,6,Solo,Turtle Rock,[followgrubby],[koza],[followgrubby]
6,2019-02-02 22:30:00,79844780,13,Solo,Terenas Stand_ L V,[followgrubby],[IamTry],[followgrubby]
7,2019-02-02 22:15:00,79844681,11,Solo,Twisted Meadows,[followgrubby],[tonbegali],[followgrubby]
8,2019-02-02 20:06:00,79843923,19,Solo,Echo Isles,[followgrubby],[FoxyDi],[followgrubby]
9,2018-12-04 02:07:00,79824361,0,Solo,Amazonia,[followgrubby],[Calanderboy],[followgrubby]


In [11]:
page = 1
data_all = []

while True:
    history_page = HistoryPage('followgrubby', 'northrend', page)
    data = list(history_page.games())
    data_all.extend(data)
    next_page = history_page.next_page

    if page >= next_page:
        break
    page = next_page


In [12]:
df = pd.DataFrame(data_all)
df

,date,game_id,game_length,game_type,map,team_one,team_two,winner
0,2019-02-03 01:22:00,79845695,11,Solo,Twisted Meadows,[followgrubby],[pwarrior],[pwarrior]
1,2019-02-03 01:08:00,79845644,12,Solo,Concealed Hill,[followgrubby],[Uberelf],[followgrubby]
2,2019-02-03 00:52:00,79845577,12,Solo,Last Refuge,[followgrubby],[atm],[followgrubby]
3,2019-02-03 00:37:00,79845522,9,Solo,Northern Isles,[followgrubby],[Critical],[followgrubby]
4,2019-02-02 22:57:00,79844958,10,Solo,Twisted Meadows,[followgrubby],[Alchh],[followgrubby]
5,2019-02-02 22:40:00,79844851,6,Solo,Turtle Rock,[followgrubby],[koza],[followgrubby]
6,2019-02-02 22:30:00,79844780,13,Solo,Terenas Stand_ L V,[followgrubby],[IamTry],[followgrubby]
7,2019-02-02 22:15:00,79844681,11,Solo,Twisted Meadows,[followgrubby],[tonbegali],[followgrubby]
8,2019-02-02 20:06:00,79843923,19,Solo,Echo Isles,[followgrubby],[FoxyDi],[followgrubby]
9,2018-12-04 02:07:00,79824361,0,Solo,Amazonia,[followgrubby],[Calanderboy],[followgrubby]


In [ ]:
df.columns

In [ ]:
"""Scrapes a WC3 profile page from battlenet and returns json."""
from bs4 import BeautifulSoup
from botocore.vendored import requests
import dateparser
import pandas as pd
from config import data_positions


class Profile:
    def __init__(self, player, server):
        self.player = player
        self.server = server.title()
        self.params = {'PlayerName': self.player, 'Gateway': self.server}
        self.url = 'http://classic.battle.net/war3/ladder/w3xp-player-profile.aspx?'
        self.servers = ['azeroth', 'lordaeron', 'northrend', 'kalimdor']
        self.soup = self.get_soup()
        self._validate()
        self.tables = self._parse_tables()

    def __str__(self):
        return '{}@{}'.format(self.player, self.server)

    def get_soup(self):
        try:
            r = requests.get(self.url, params=self.params)
        except requests.exceptions.RequestException as e:
            print(e)

        return BeautifulSoup(r.content, 'lxml')

    def _validate(self):
        self.validate_server()
        self.validate_player()

    def parse(self):
        data = {}
        data['info'] = self.information
        data['individual'] = self.individual
        data['team'] = self.team
        return data

    def _parse_tables(self):
        soup = self.soup.find('table', class_='mainTable')
        tables = soup.find_all('td', {'align': 'center', 'valign': 'top'})
        tables = dict(zip(['info', 'individual', 'team'], tables))
        return tables

    @property
    def information(self):
        return {
            'player': self.player,
            'server': self.server,
            'clan': self.clan,
            'main_race': self.main_race,
            'home_page': self.home_page,
            'additional_info': self.parse_additional_info,
            'last_ladder_game': self.last_ladder_game
        }

    @property
    def individual(self):
        soup = self.tables.get('individual')
        type_ = 'individual'
        vocab = {'Team Games': 'random_team', 'Solo Games': 'solo', 'FFA Games': 'free_for_all'}
        data = {}

        for game_type, new_key in vocab.items():
            container = soup.find(text=game_type)
            if container is not None:
                table = container.parent.parent.parent.parent.parent
                values = [x.get_text() for x in table.find_all('b')]
                d = self.format_values(type_, values)
                d['win_percentage'] = self.calc_win_percentage(d['wins'], d['losses'])
                new_key = vocab[game_type]
                data[new_key] = d

        return data

    @property
    def team(self):
        soup = self.tables.get('team')
        type_ = 'teams'
        teams = soup.find_all(text='Partner(s):')
        data = []

        for team in teams:
            table = team.parent.parent.parent.parent.parent.parent.parent.parent
            values = self.extract_values(table)
            d = self.format_values(type_, values)
            d['win_percentage'] = self.calc_win_percentage(d['wins'], d['losses'])
            data.append(d)

        return data

    @property
    def home_page(self):
        soup = self.tables.get('info')
        home_page = soup.find('div', {'id': 'homePage'})
        home_page = home_page.b.get_text().strip()
        if home_page == '':
            return 'N/A'
        return home_page

    @property
    def parse_additional_info(self):
        soup = self.tables.get('info')
        additional_info_div = soup.find('div', {'id': 'additionalInfo'})
        script = additional_info_div.script.get_text()
        text_start = 'document.write("'
        i = script.find(text_start)
        if i == -1:
            additional_info = None
        else:
            i += len(text_start)

        text_end = '");'
        j = script[i:].find(text_end)
        additional_info = script[i: i + j]

        if additional_info in ['', None]:
            return 'N/A'
        return additional_info

    @property
    def last_ladder_game(self):
        soup = self.tables.get('info')
        last_ladder_game = soup.find(text='Last Ladder Game:')
        if last_ladder_game is not None:
            last_ladder_game = last_ladder_game.parent.parent.b.get_text()
        last_ladder_game = str(dateparser.parse(last_ladder_game).strftime('%Y-%m-%d'))
        return last_ladder_game

    @property
    def main_race(self):
        soup = self.tables.get('info')
        overall_stats_table = soup.find('td', class_='rankingHeader')
        if overall_stats_table is not None:
            overall_stats_table = overall_stats_table.parent.parent
        rows = overall_stats_table.find_all('tr')[1:-1]
        df = self.parse_stats_table(rows)
        if len(df[df['percentage_games'] >= 75]) != 0:
            main_race = df[df['percentage_games'] >= 75]['race'].values[0].title()
        else:
            main_race = 'No main race'

        return main_race

    @staticmethod
    def parse_stats_table(rows):
        data = []
        keys = ['race', 'wins', 'losses', 'win_percentage']

        for row in rows:
            row = [x.get_text().strip() for x in row.find_all('td')]
            row = dict(zip(keys, row))
            row['num_games'] = int(row['losses']) + int(row['wins'])
            data.append(row)

        df = pd.DataFrame(data)
        df['percentage_games'] = df['num_games'] * 100 / sum(df['num_games'])
        df['race'] = df['race'].apply(lambda x: x.lower().replace(':', ''))

        for col in ['wins', 'losses']:
            df[col] = df[col].astype(int)

        df['total_games'] = df['losses'] + df['wins']

        return df

    @property
    def clan(self):
        soup = self.tables.get('info')
        clan_url = soup.find('a', href=lambda x: x and x.startswith('w3xp-clan-profile.aspx?'))
        if clan_url is not None:
            return clan_url.get_text()
        return 'N/A'

    @staticmethod
    def format_values(type_, values):
        fields = ['wins', 'losses', 'partners', 'level', 'rank', 'experience']
        data = {}

        for field in fields:
            meta_data = data_positions[type_][field]
            if not meta_data:
                continue
            i = meta_data['position']
            v = values[i]
            if not v:
                continue
            formatter = meta_data['function']
            if formatter:
                value = formatter(v)
            else:
                value = v

            data[field] = value

        return data

    @staticmethod
    def calc_win_percentage(wins, losses):
        win_percentage = round((100 * int(wins)) / (int(wins) + int(losses)), 2)
        return '{0:.2f}%'.format(win_percentage)

    @staticmethod
    def extract_values(table):
        values = []
        values_old = table.find_all('b')

        for i, value in enumerate(values_old):
            if i == 3:
                partners = [x.get_text() for x in value]
                if len(partners) > 1:
                    partners.remove('')
                values.append(partners)
            else:
                values.append(value.get_text())

        return values

    def validate_player(self):
        error_span = self.soup.find('span', class_='colorRed')
        if error_span is not None:
            raise Exception('{} | Profile not found'.format(str(self)))

    def validate_server(self):
        if self.server.lower() not in self.servers:
            raise Exception('{} | Invalid server'.format(str(self)))

    def request_solo(self):
        data = self.individual.get('solo')
        if data is not None:
            data.update({
                'main_race': self.main_race,
                'player': self.player,
                'server': self.server,
            })
        return data

    def request_random_team(self):
        data = self.individual.get('random_team')
        if data is not None:
            data.update({
                'main_race': self.main_race,
                'player': self.player,
                'server': self.server,
            })
        return data

    def request_info(self):
        data = self.information
        if data is not None:
            return data

if __name__ == '__main__':
    players = [
        {
            'player': 'romantichuman',
            'server': 'northrend'
        },
        {
            'player': 'pieck',
            'server': 'northrend'
        },
        {
            'player': 'pieck',
            'server': 'azeroth'
        },
        {
            'player': 'wearefoals',
            'server': 'northrend'
        },
        {
            'player': 'wearefoals',
            'server': 'azeroth'
        },
        {
            'player': 'wearefoals',
            'server': 'fake_server'
        },
        {
            'player': 'fake_player',
            'server': 'azeroth'
        },
        {
            'player': 'moon1234',
            'server': 'northrend'
        },
    ]
    print('-- Testing --')
    for player in players:
        print()
        try:
            profile = Profile(**player)
            print(profile)
            print(profile.parse())
        except Exception as e:
            print(e)
